<a href="https://colab.research.google.com/github/AlishbaNazir/GoodNotes/blob/main/Updated_GoodNotes_notes_taking_app.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!apt install ffmpeg

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 29 not upgraded.


In [2]:
# Install required libraries
!pip install openai-whisper spacy dateparser transformers scikit-learn

# Import libraries
import whisper
import spacy
import numpy as np
from dateparser import parse
from transformers import pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from google.colab import files

In [3]:
# Download and load SpaCy NLP model
spacy.cli.download("en_core_web_sm")
nlp = spacy.load("en_core_web_sm")

# Load Whisper model
model = whisper.load_model("base")

# Load Summarization model
summarizer = pipeline("summarization")

def transcribe_audio(file_path):
    """Transcribe audio using Whisper AI"""
    result = model.transcribe(file_path)
    return result["text"]

def extract_deadlines_and_notes(text):
    """Extract deadlines and key notes from text"""
    doc = nlp(text)
    deadlines = []
    key_notes = []

    # Extract deadlines (dates & times)
    for ent in doc.ents:
        if ent.label_ in ["DATE", "TIME"]:
            parsed_date = parse(ent.text)
            if parsed_date:
                deadlines.append(parsed_date.strftime("%Y-%m-%d"))

    # Extract Key Notes Using TF-IDF Instead of NLTK
    sentences = text.split(". ")  # Simple sentence splitting
    if len(sentences) > 5:
        vectorizer = TfidfVectorizer()
        tfidf_matrix = vectorizer.fit_transform(sentences)
        sentence_scores = np.sum(tfidf_matrix.toarray(), axis=1)
        top_sentences = [sentences[i] for i in np.argsort(sentence_scores)[-3:]]  # Get top 3 sentences
        key_notes = " ".join(top_sentences)
    else:
        key_notes = text  # If short text, return as is

    return deadlines, key_notes

def main():
    # Upload an audio file
    print("Please upload an audio file (e.g., .wav or .mp3)")
    uploaded = files.upload()

    # Get the uploaded file path
    file_path = list(uploaded.keys())[0]

    # Transcribe the audio
    print("\nTranscribing Audio...")
    transcription = transcribe_audio(file_path)
    print("\nExtracted Text:\n", transcription)

    # Extract deadlines and key notes
    print("\nExtracting Deadlines and Key Notes...")
    deadlines, summary = extract_deadlines_and_notes(transcription)

    # Display structured output
    print("\n🔹 Deadlines Found:", deadlines)
    print("\n📝 Key Notes:", summary)

if __name__ == "__main__":
    main()


✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


/usr/local/lib/python3.11/dist-packages/whisper/__init__.py:150: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(fp, map_location=device)
No model was 

Please upload an audio file (e.g., .wav or .mp3)


Saving ab.unknown to ab (1).unknown

Transcribing Audio...

Extracted Text:
  I can note that I am having assignment view of generative AI at 14 of March.

Extracting Deadlines and Key Notes...

🔹 Deadlines Found: ['2025-03-14']

📝 Key Notes:  I can note that I am having assignment view of generative AI at 14 of March.


User InterFace

In [4]:
!pip install gradio


In [6]:
import gradio as gr
import whisper
import tempfile

# Load Whisper model
model = whisper.load_model("base")

def transcribe_audio(audio_file):
    # Save audio to a temp file
    with tempfile.NamedTemporaryFile(delete=False, suffix=".mp3") as temp:
        temp.write(audio_file)
        temp_path = temp.name

    # Transcribe audio
    result = model.transcribe(temp_path)

    return result["text"]

# Create Gradio app
app = gr.Interface(
    fn=transcribe_audio,
    inputs=gr.Audio(type="filepath"),  # FIXED LINE
    outputs="text",
    title="GoodNotes - AI-Powered Transcription"
)

# Launch the app
app.launch(share=True)


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://514a68d09a7aee68dd.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
